# Principal Component Analysis

## Task

- In the Portland Housing Prices/Sales dataset Jul 2020 - Jul 2021, determine if there is a multicollinearity problem.
- Perform a principal component analysis
- Create a linear model from PCA and compare the model to the previous exercise

# Data loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_csv ("../dataset/portland_filtered.csv",  sep=';')

# Analysis
- show data
- solve NaN values
- display basic data statistics
- show correlation matrix

In [ ]:
fig = plt.figure(figsize=(8, 6))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data.head()

# Data preparation
- select X variables bathrooms', 'bedrooms', 'livingArea','age','price'
- standardize the variables

In [ ]:
def rescale(X):
    mean = X.mean()
    std = X.std()
    scaled_X = [(i - mean) / std for i in X]
    return pd.Series(scaled_X)

data_std = pd.DataFrame()
for col in data.columns:
    data_std[col] = rescale(data[col])

print("Standardized data statistics:")
data_std.describe()

In [ ]:
features = ['bathrooms', 'bedrooms', 'livingArea', 'age']
X = data[features]
y = data['price']

print("Features:")
print(X.head())

# Display the VIF for each variable

In [ ]:
vifdf = []
for i in data.columns:
    X_vif = np.array(data.drop(i, axis=1))
    y_vif = np.array(data[i])
    lr = LinearRegression()
    lr.fit(X_vif, y_vif)
    y_pred = lr.predict(X_vif)
    r2 = r2_score(y_vif, y_pred)
    vif = 1 / (1 - r2)
    vifdf.append((i, vif))

vifdf = pd.DataFrame(vifdf, columns=['Features', 'Variance Inflation Factor'])
vifdf.sort_values(by='Variance Inflation Factor')

# PCA
- Perform PCA
- show correlation matrix

In [ ]:
fig = plt.figure(figsize=(8, 6))
sns.heatmap(data_std_pca.corr(), annot=True, cmap='coolwarm')
plt.title("PCA Components Correlation Matrix")
plt.show()

In [ ]:
data_std_pca = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4'])
data_std_pca['price'] = data_std['price'].values

print("PCA data head:")
data_std_pca.head()

In [ ]:
pca = PCA(n_components=4)
X_std = data_std[features]
X_pca = pca.fit_transform(X_std)

print(f"Original shape: {X_std.shape}")
print(f"PCA shape: {X_pca.shape}")
print(f"\nExplained variance ratio: {pca.explained_variance_ratio_}")
print(f"Cumulative explained variance: {np.cumsum(pca.explained_variance_ratio_)}")

# Linear model
- Create and train a Linear Model for PCA variables
- Show R2 and RMSE

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].scatter(y_test_orig, y_test_pred_orig, alpha=0.6, color='blue')
axes[0].plot([y_test_orig.min(), y_test_orig.max()], [y_test_orig.min(), y_test_orig.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Price')
axes[0].set_ylabel('Predicted Price')
axes[0].set_title(f'Original Features\nR2: {test_r2_orig:.4f}, RMSE: {test_rmse_orig:.4f}')
axes[0].grid(True)

axes[1].scatter(y_test, y_test_pred, alpha=0.6, color='green')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Price')
axes[1].set_ylabel('Predicted Price')
axes[1].set_title(f'PCA Components\nR2: {test_r2:.4f}, RMSE: {test_rmse:.4f}')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 70)
print("COMPARISON: Original Features vs PCA Components")
print("=" * 70)

comparison_df = pd.DataFrame({
    'Model': ['Original Features', 'PCA Components'],
    'Features': [4, 4],
    'Train R2': [train_r2_orig, train_r2],
    'Train RMSE': [train_rmse_orig, train_rmse],
    'Test R2': [test_r2_orig, test_r2],
    'Test RMSE': [test_rmse_orig, test_rmse]
})

print(comparison_df.to_string(index=False))
print("=" * 70)
print("\nKey Observations:")
print("- PCA removes multicollinearity between features")
print("- Both models should show similar performance")
print("- PCA components are uncorrelated (orthogonal)")

In [ ]:
# Train model on original standardized features for comparison
X_orig = data_std[features].values
y_orig = data_std['price'].values

X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_orig, y_orig, test_size=0.25, random_state=42)

lr_orig = LinearRegression()
lr_orig.fit(X_train_orig, y_train_orig)

y_train_pred_orig = lr_orig.predict(X_train_orig)
y_test_pred_orig = lr_orig.predict(X_test_orig)

train_r2_orig = r2_score(y_train_orig, y_train_pred_orig)
train_rmse_orig = np.sqrt(mean_squared_error(y_train_orig, y_train_pred_orig))
test_r2_orig = r2_score(y_test_orig, y_test_pred_orig)
test_rmse_orig = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_orig))

print("Original features model (no PCA):")
print(f"Training - R2: {train_r2_orig:.4f}, RMSE: {train_rmse_orig:.4f}")
print(f"Test - R2: {test_r2_orig:.4f}, RMSE: {test_rmse_orig:.4f}")

# Comparison with previous linear regression model

In [ ]:
y_test_pred = lr_pca.predict(X_test)
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Test set performance:")
print(f"R2 score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}")

In [ ]:
y_train_pred = lr_pca.predict(X_train)
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

print("Training set performance:")
print(f"R2 score: {train_r2:.4f}")
print(f"RMSE: {train_rmse:.4f}")

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_train, y_train)

print("Model coefficients:")
print(f"Intercept: {lr_pca.intercept_}")
print(f"Coefficients: {lr_pca.coef_}")

In [ ]:
X_pca_full = np.array(data_std_pca.drop('price', axis=1))
y_pca = np.array(data_std_pca['price'])

X_train, X_test, y_train, y_test = train_test_split(X_pca_full, y_pca, test_size=0.25, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")